In [15]:
import time
import pandas as pd
from datetime import datetime
import threading

import bernielib as bl

bl.listSerialPorts()

[]

In [2]:
ber = bl.robot(cartesian_port_name='COM18', pipette_port_name='COM7', misc_port_name='COM20')

In [3]:
ber.robotHome()
ber.pipetteHome()

# Refill tips

In [4]:
ber.tips_rack.refill()

# Recalibrate pipette tip rack

In [5]:
ber.calibrateRack(rack='tips')

(76.80000000000001, 149.765, 168.7)

In [8]:
ber.move(z=50)

# Preparing DNA for the experiment

## Preparing fake DNA sample, only for testing. Don't run at a real run

In [6]:
dna_500 = bl.createSample('eppendorf', 'dna_500', ber.samples_rack, 0, 10, 0)
water = bl.createSample('50ml', 'water', ber.reagents_rack, 1, 1, 50000)

In [9]:
ber.pickUpNextTip()

In [10]:
ber.move(z=50)

In [11]:
ber.transferLiquid(water, dna_500, 50)

In [12]:
ber.move(z=50)

In [13]:
ber.dumpTipToWaste()

## Preparing diluted stock solution

In [5]:
dna_500 = bl.createSample('eppendorf', 'dna_500', ber.samples_rack, 0, 10, 50)
dna_stock = bl.createSample('eppendorf', 'dna_stock', ber.samples_rack, 0, 9, 0)
water = bl.createSample('50ml', 'water', ber.reagents_rack, 1, 1, 50000)

In [6]:
ber.pickUpNextTip()

In [7]:
ber.move(z=50)

In [8]:
ber.transferLiquid(water, dna_stock, 190)

In [9]:
ber.transferLiquid(dna_500, dna_stock, 10, v_immerse_dispense=1600)

In [10]:
script = pd.read_csv('mixing_pattern_eppendorf.csv')

In [11]:
ber.mixByScript(dna_stock, script)

In [12]:
ber.pipetteHome()

## Preparing sample solutions with DNA ladder

In [13]:
samples_list = bl.createSamplesToPurifyList(ber, [0,0,0,0,0,0])

In [14]:
for sample in samples_list:
    ber.transferLiquid(dna_stock, sample, 30, v_immerse_dispense=1600)

In [15]:
ber.move(z=50)

In [16]:
ber.dumpTipToWaste()

In [17]:
ber.pipetteHome()

# Running a protocol

In [4]:
def getSampleTopZ(sample, rack):
    z_above_rack = sample.getZAboveRack()
    z_rack_top = rack.getZ()
    return z_rack_top - z_above_rack

In [5]:
def calcBeadsVolumeToAdd(robot, samples_list, cutoff_list):
    v_beads_list = []
    for sample, dna_size in zip(samples_list, cutoff_list):
        v_beads = robot.calcBeadsVol(sample, dna_size)
        v_beads_list.append(v_beads)
    return v_beads_list

In [6]:
def waitAfterTimestamp(timestamp, delay):
    new_ts = time.time()
    while (new_ts - timestamp) < delay:
        time.sleep(1)
        new_ts = time.time()

In [7]:
def addBeads(robot, sample, beads, v_beads, sample_mix_scenario, beads_mix_scenario, z_safe=50):
    robot.move(z=z_safe)
    #robot.pickUpNextTip()
    #robot.move(z=z_safe)
    # Mixing beads before experiment
    #robot.mixByScript(beads, beads_mix_scenario)
    # Transferring beads to sample
    robot.transferLiquid(beads, sample, v_beads)
    # Mixing sample with beads
    robot.mixByScript(sample, sample_mix_scenario)
    robot.move(z=z_safe)
    #robot.dumpTipToWaste()
    robot.move(z=z_safe)

In [8]:
def addBeadsToAll(robot, samples_list, v_beads_list, beads, sample_mix_scenario, beads_mix_scenario):
    robot.moveMagnetsAway(poweroff=True)
    robot.pickUpNextTip()
    robot.move(z=50)
    robot.mixByScript(beads, beads_mix_scenario)
    robot.move(z=50)
    
    for sample, v_beads in zip(samples_list, v_beads_list):
        robot.transferLiquid(beads, sample, v_beads)
    
    counter = 0
    for sample, v_beads in zip(samples_list, v_beads_list):
        if counter != 0:
            robot.move(z=50)
            robot.pickUpNextTip()
        
        robot.move(z=50)
        
        robot.mixByScript(sample, sample_mix_scenario)
            
        if counter == 0:
            timestamp = time.time()
        counter += 1
        
        robot.move(z=50)
        robot.dumpTipToWaste()
    
    return timestamp

In [20]:
def removeSupernatant(robot, sample, waste, z_safe=50, delay=0.5):
    robot.pickUpNextTip()
    robot.move(z=z_safe)
    z0 = robot._getTubeZBottom(sample)
    while sample.getVolume() > 200:
        robot.movePipetteToVolume(200)
        robot.moveToSample(sample)
        robot.move(z=z0-0.5)
        robot.movePipetteToVolume(0)
        new_vol = sample.getVolume() - 200
        if new_vol < 0:
            new_vol = 0
        sample.setVolume(new_vol)
        robot.move(z=z_safe)
        robot.dispenseLiquid(waste, 200, blow_extra=True)
        robot.move(z=z_safe)
    pipetteThread = threading.Thread(target=robot.movePipetteToVolume, args=(250,))
    pipetteThread.start()
    robot.moveToSample(sample)
    #robot.movePipetteToVolume(250)
    robot.move(z=z0)
    pipetteThread.join()
    robot.movePipetteToVolume(50)
    time.sleep(delay)
    robot.move(z=z0-0.5)
    time.sleep(delay)
    robot.moveAxisDelta('X', 1.5)
    robot.movePipetteToVolume(40)
    time.sleep(delay/4.0)
    robot.moveAxisDelta('X', -1.5)
    robot.moveAxisDelta('Y', 1.5)
    robot.movePipetteToVolume(30)
    time.sleep(delay/4.0)
    robot.moveAxisDelta('Y', -1.5)
    robot.moveAxisDelta('X', -1.5)    
    robot.movePipetteToVolume(20)
    time.sleep(delay/4.0)
    robot.moveAxisDelta('X', 1.5)
    robot.moveAxisDelta('Y', -1.5)
    robot.movePipetteToVolume(0)
    time.sleep(delay/4.0)
    robot.move(z=z_safe)
    robot.dispenseLiquid(waste, 250, blow_extra=True)
    robot.move(z=z_safe)
    robot.dumpTipToWaste()

In [26]:
def removeSupernatantFast(robot, sample, waste, z_safe=50, delay=0.5):
    robot.pickUpNextTip()
    robot.move(z=z_safe)
    z0 = robot._getTubeZBottom(sample)
    while sample.getVolume() > 0:
        pipetteThread = threading.Thread(target=robot.movePipetteToVolume, args=(200,))
        pipetteThread.start()
        robot.moveToSample(sample)
        robot.move(z=z0-0.5)
        pipetteThread.join()
        robot.movePipetteToVolume(0)
        new_vol = sample.getVolume() - 200
        if new_vol < 0:
            new_vol = 0
        sample.setVolume(new_vol)
        time.sleep(delay)
        robot.move(z=z_safe)
        robot.dispenseLiquid(waste, 200, blow_extra=True)
        robot.move(z=z_safe)
    robot.dumpTipToWaste()

In [10]:
s = bl.createSamplesToPurifyList(ber, [0])[0]
water = bl.createSample('50ml', 'water', ber.reagents_rack, 1, 1, 50000)
waste = bl.createSample('50ml', 'liquid_waste', ber.reagents_rack, 0, 1, 0)

In [27]:
ber.pickUpNextTip()

In [28]:
ber.transferLiquid(water, s, 200)

In [29]:
ber.move(z=50)

In [32]:
ber.dumpTipToWaste()

In [31]:
print(datetime.now().strftime("%H:%M:%S"))
removeSupernatant(ber, s, waste)
print(datetime.now().strftime("%H:%M:%S"))

11:53:13


KeyboardInterrupt: 

In [ ]:
# Baseline with probing the bottom: 1 min 7 sec
# Baseline as in the experiment: 1 min 1 sec
# Uptake everything once, mopping up immediately: 0 min 42 sec
# Removed pressure sensors zeroing before tip heavy pickup: 0 min 37 sec
# Removed zeroing also before fine approach: 0 min 37 sec
# Fine tuning tip pickup: 0 min 37 sec
# Plunger moves down simultaneously with the XYZ: 0 min 35 sec
# Fast removal, without mopping liquid from corners: 0 min 30 sec

In [33]:
print(datetime.now().strftime("%H:%M:%S"))
removeSupernatantFast(ber, s, waste)
print(datetime.now().strftime("%H:%M:%S"))

11:53:31
11:54:01


In [22]:
ber.tips_rack.getZ()

169.4

In [28]:
ber.tips_rack.calcWellXY(8,4)

(99.625, 154.41000000000003)

In [29]:
ber.move(99.6, 154)

In [34]:
ber.move(z=169.4-12.5)

In [14]:
ber.dumpTipToPosition(0,4)

In [9]:
def removeSupernatantAllSamples(robot, samples_list, waste):
    counter = 0
    for sample in samples_list:
        removeSupernatant(robot, sample, waste)
        if counter == 0:
            sample_dried_timestamp = time.time()
        counter += 1
    return sample_dried_timestamp

In [10]:
def add80PctEthanol(robot, samples_list, ethanol, volume, z_safe=50):
    robot.pickUpNextTip()
    robot.move(z=z_safe)
    
    counter = 0
    for sample in samples_list:
        robot.transferLiquid(ethanol, sample, volume, touch_wall=False)
        if counter == 0:
            ethanol_added_time = time.time()
        counter += 1
    
    robot.move(z=z_safe)
    robot.dumpTipToWaste()
    
    return ethanol_added_time

In [76]:
def elutionMix(robot, sample, volume, delay=0.5):
    z0 = robot._getTubeZBottom(sample)
    z_top = sample.getSampleTopAbsZ(added_length=robot._calcExtraLength())
    robot.movePipetteToVolume(0)
    robot.movePipetteToVolume(volume+5)
    robot.movePipetteToVolume(volume)
    robot.move(z=z0-0.5)
    robot.movePipetteToVolume(0)
    time.sleep(delay)
    # Washing steps, moving along the wall
    # 1
    z_curr = z_top + 24
    #z_curr = sample.calcAbsLiquidLevelFromVol(500, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', 3.0)
    robot.movePipetteToVolume(volume/4.0)
    time.sleep(delay/4.0)
    # 2
    z_curr = z_top + 28
    #z_curr = sample.calcAbsLiquidLevelFromVol(300, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    #robot.moveAxisDelta('X', -0.629)
    robot.movePipetteToVolume(2 * (volume/4.0))
    time.sleep(delay/4.0)
    # 3
    z_curr = z_top + 32
    #z_curr = sample.calcAbsLiquidLevelFromVol(150, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', -0.629)
    robot.movePipetteToVolume(3 * (volume/4.0))
    time.sleep(delay/4.0)
    # 4
    z_curr = z_top + 36
    #z_curr = sample.calcAbsLiquidLevelFromVol(150, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', -0.629)
    robot.movePipetteToVolume(volume)
    time.sleep(delay/4.0)
    
    x, y = sample.getCenterXY()
    robot.move(x=x, y=y)
    robot.movePipetteToVolume(volume+50)
    
    z_curr = sample.calcAbsLiquidLevelFromVol(volume+100, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.movePipetteToVolume(0)

In [173]:
def elutionMixLrgVol(robot, sample, volume, delay=0.5):
    z0 = robot._getTubeZBottom(sample)
    z_top = sample.getSampleTopAbsZ(added_length=robot._calcExtraLength())
    
    # Uptaking
    robot.movePipetteToVolume(0)
    robot.movePipetteToVolume(volume+5)
    robot.movePipetteToVolume(volume)
    robot.move(z=z0-0.8)
    robot.movePipetteToVolume(0)
    time.sleep(delay)
    
    # Ejecting liquid
    z_curr = sample.calcAbsLiquidLevelFromVol(1000, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', 3.4)
    robot.movePipetteToVolume(volume+50)
    z_curr = sample.calcAbsLiquidLevelFromVol(500, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    
    # To back position
    x, y = sample.getCenterXY()
    robot.move(x=x, y=y)
    
    robot.movePipetteToVolume(0)

In [136]:
s = bl.createSamplesToPurifyList(ber, [30])[0]

In [104]:
ber.pickUpNextTip()

In [189]:
ber.move(z=50)

In [52]:
x, y = s.getCenterXY()

In [53]:
ber.move(x=x, y=y)

In [85]:
elutionMix(ber, s, 30)

In [186]:
elutionMixLrgVol(ber, s, 30, delay=1.0)

In [41]:
ber.dumpTipToWaste()

In [50]:
water = bl.createSample('50ml', 'water', ber.reagents_rack, 1, 1, 50000)

In [180]:
ber.transferLiquid(water, s, 30)

In [61]:
ber.movePipetteToVolume(0)

In [147]:
s.setVolume(500)

In [178]:
ber.transferLiquid(s, waste, 500)

In [179]:
ber.moveMagnetsAway(poweroff=True)

In [177]:
ber.moveMagnetsTowardsTube(poweroff=True)

In [192]:
ber.dumpTipToWaste()

In [188]:
#TODO: make safe_z a general property of the robot
def elute(robot, sample, eluent, volume, mix_delay=0.5, mix_times=8, safe_z=50):
    robot.moveMagnetsAway(poweroff=True)
    robot.pickUpNextTip()
    robot.move(z=safe_z)
    robot.transferLiquid(eluent, sample, volume)
    for i in range(mix_times):
        elutionMixLrgVol(robot, sample, volume)
    elutionMix(robot, sample, volume)
    elutionMix(robot, sample, volume)
    elution_start_time = time.time()
    robot.move(z=safe_z)
    robot.dumpTipToWaste()
    return elution_start_time

def eluteAllSamples(robot, samples_list, eluent, V_eluent, mix_delay=0.5, mix_times=6, safe_z=50):
    counter = 0
    for sample in samples_list:
        ts = elute(ber, sample, eluent, V_eluent, mix_delay=mix_delay, mix_times=mix_times, safe_z=safe_z)
        print ()
        if counter == 0:
            elution_start_timestamp = ts
        counter += 1
    return elution_start_timestamp

In [14]:
def separateEluate(robot, eluate_tube, result_tube, pipette_delay=0.5, z_above_bottom=0.5, safe_z=50):
    robot.pickUpNextTip()
    robot.move(z=safe_z)
    # Uptaking liquid
    v = eluate_tube.getVolume()
    robot.moveToSample(eluate_tube)
    robot.movePipetteToVolume(v+5)
    robot.movePipetteToVolume(v)
    z0 = robot._getTubeZBottom(eluate_tube)
    robot.move(z=z0-z_above_bottom)
    robot.movePipetteToVolume(0)
    time.sleep(pipette_delay)
    # Dispensing liquid
    robot.moveToSample(result_tube)
    robot.movePipetteToVolume(v+5)
    time.sleep(pipette_delay)
    robot.movePipetteToVolume(200)
    time.sleep(pipette_delay)
    robot.touchWall(result_tube)
    robot.movePipetteToVolume(0)
    robot.move(z=safe_z)
    ber.dumpTipToWaste()

In [15]:
def oneStagePurification(robot, N_samples, initial_vol_list, cutoff_list, V_avail_beads, V_avail_water, V_avail_ethanol, 
                         beads_rack, beads_col, beads_row, sample_mix_scenario, beads_mix_scenario,
                         V_waste=0, T_absorb=600, T_pull=60, T_wash=30, T_dry=300, T_elute=600,
                         V_wash=200, V_elute=30):
    
    samples_list = bl.createSamplesToPurifyList(robot, initial_vol_list)
    result_list = bl.createPurifiedSamplesList(robot, N_samples)
    beads = bl.createSample('eppendorf', 'beads', beads_rack, beads_col, beads_row, V_avail_beads)
    waste = bl.createSample('50ml', 'liquid_waste', robot.reagents_rack, 0, 1, V_waste)
    water = bl.createSample('50ml', 'water', robot.reagents_rack, 1, 1, V_avail_water)
    EtOH80pct = bl.createSample('50ml', 'EtOH80pct', robot.reagents_rack, 2, 1, V_avail_ethanol)
    
    v_beads_list = calcBeadsVolumeToAdd(robot, samples_list, cutoff_list=cutoff_list)
    print( )
    # Adding beads
    timestamp_beads_added = addBeadsToAll(robot, samples_list, v_beads_list, beads, sample_mix_scenario, beads_mix_scenario)
    waitAfterTimestamp(timestamp_beads_added, T_absorb)
    # Removing supernatant
    robot.moveMagnetsTowardsTube(poweroff=True)
    time.sleep(T_pull)
    ts = removeSupernatantAllSamples(robot, samples_list, waste)
    # Ethanol wash
    # Wash 1
    timestamp_ethanol_added = add80PctEthanol(robot, samples_list, EtOH80pct, V_wash)
    waitAfterTimestamp(timestamp_ethanol_added, T_wash)
    ts = removeSupernatantAllSamples(robot, samples_list, waste)
    # Wash 2
    timestamp_ethanol_added = add80PctEthanol(robot, samples_list, EtOH80pct, V_wash)
    waitAfterTimestamp(timestamp_ethanol_added, T_wash)
    timestamp_ethanol_removed = removeSupernatantAllSamples(robot, samples_list, waste)
    # Drying ethanol
    waitAfterTimestamp(timestamp_ethanol_removed, T_dry)
    
    # Elution
    # Adding water
    elution_start_timestamp = eluteAllSamples(robot, samples_list, water, V_elute, mix_delay=0.5, mix_times=6, safe_z=50)
    waitAfterTimestamp(elution_start_timestamp, T_elute) # elution wait
    
    # Magnetic beads to the side of the tubes
    ber.moveMagnetsTowardsTube()
    time.sleep(T_pull)
    # Moving liquid to the resulting tubes
    for sample, result in zip(samples_list, result_list):
        separateEluate(ber, sample, result)

In [16]:
script = pd.read_csv('mixing_pattern_eppendorf.csv')

In [31]:
oneStagePurification(ber, N_samples=6, initial_vol_list=[30, 30, 30, 30, 30, 30], 
                     cutoff_list=[150, 250, 350, 450, 550, 1000],
                     V_avail_beads=1000, V_avail_water=50000, V_avail_ethanol=25000, 
                     beads_rack=ber.samples_rack, beads_col=0, beads_row=11, 
                     sample_mix_scenario=script, beads_mix_scenario=script)

# Results 12/20/2020

Alex performed bioanalyzing; check results in email.

Conclusions:
1. DNA control has too high concentration. Reduce at least 10 times or more.
2. Does not differentiate on DNA length. Check beads volumes.
3. Concentration of DNA is inconsistent between tubes. Reasons:
    a). Beads over-dried; check drying time is calculated properly
    b). Elution didn't remove all beads from walls properly, modify elution program
    c). Initial mixing done badly.
    
Regarding binding capacity: 1ul of AmpureXP will bind over 3ug DNA
I used minimum 15 uL; means I can absorb 45 ug of DNA. 

I used 10 uL of 500 ng/uL for all samples; for one sample 10/7 ~ 1.2 uL; I have about 700-900 ng of DNA in each sample.

## Checking beads volumes

In [ ]:
samples_list = bl.createSamplesToPurifyList(ber, [30, 30, 30, 30, 30, 30])

In [32]:
calcBeadsVolumeToAdd(ber, samples_list, cutoff_list=[150, 250, 350, 450, 550, 1000])

[47.34245107533334,
 26.1546280812,
 20.43851894167347,
 18.135151086592593,
 16.993761418561984,
 15.4552048488]

In [33]:
ber.getBeadsVolumeCoef()

(0.499325349, -9.91043764, 25758.5836)

In [4]:
water = bl.createSample('50ml', 'water', ber.reagents_rack, 1, 1, 50000)

In [5]:
ber.pickUpNextTip()

In [6]:
ber.move(z=50)

In [7]:
ber.uptakeLiquid(sample=water, volume=15.45)

In [8]:
ber.move(z=50)

In [9]:
ber.dispenseLiquid(sample=water, volume=15.45)

In [10]:
ber.move(z=50)

In [20]:
ber.uptakeLiquid(sample=water, volume=47.34)

In [21]:
ber.move(z=50)

In [22]:
ber.dispenseLiquid(sample=water, volume=47.34, plunger_retract=False)

In [23]:
ber.move(z=50)

In [24]:
ber.pipetteMove(1)

In [ ]:
# Lower volume seems to be visually similar to what I observed during experiment; higher volume unsure.

In [25]:
ber.dumpTipToWaste()

# Checking drying time

In [17]:
from datetime import datetime

In [22]:
samples_list = bl.createSamplesToPurifyList(ber, [200, 200, 200])
waste = bl.createSample('50ml', 'liquid_waste', ber.reagents_rack, 0, 1, 0)

In [23]:
ber.robotHome()
ber.pipetteHome()

In [24]:
print("Pipetting started ", datetime.now().strftime("%H:%M:%S"))
timestamp_ethanol_removed = removeSupernatantAllSamples(ber, samples_list, waste)
print("Pipetting finished ", datetime.now().strftime("%H:%M:%S"))
waitAfterTimestamp(timestamp_ethanol_removed, 120)
print("Wait finished ", datetime.now().strftime("%H:%M:%S"))

Pipetting started  10:35:58
Pipetting finished  10:39:23
Wait finished  10:39:23


In [33]:
300/60

5.0

In [39]:
def test():
    ts = time.time()
    time.sleep(60)
    return ts

In [40]:
start = datetime.now().strftime("%H:%M:%S")
print(start.strftime("%H:%M:%S"))
t = test()
t1 = time.time()
t1 - t
end = datetime.now()
print(end.strftime("%H:%M:%S"))

19:55:16
19:56:16


In [38]:
5*60

300

In [42]:
datetime.now().strftime("%H:%M:%S")

'20:06:21'

In [54]:
def test1(robot, samples_list, waste):
    counter = 0
    for sample in samples_list:
        #removeSupernatant(robot, sample, waste)
        time.sleep(1)
        print(datetime.now().strftime("%H:%M:%S"))
        if counter == 0:
            print("Timestamp created", datetime.now().strftime("%H:%M:%S"))
            sample_dried_timestamp = time.time()
            print(sample_dried_timestamp)
        print(counter)
        counter += 1
    return sample_dried_timestamp

In [55]:
test1(ber, samples_list, waste)

20:23:03
Timestamp created 20:23:03
1608610983.3854115
0
20:23:04
1
20:23:05
2


1608610983.3854115

In [26]:
samples_list[0].getZAboveRack()

9.6

In [27]:
ber.samples_rack.getZ()

155.3

In [29]:
getSampleTopZ(samples_list[0], ber.samples_rack)

145.70000000000002

In [32]:
samples_list[0].getCenterXY()

(193.0, 49.02499999999998)

# Purging sample tubes

In [53]:
samples_list = bl.createSamplesToPurifyList(ber, [200,200,200,200,200,200])

In [54]:
waste = bl.createSample('50ml', 'liquid_waste', ber.reagents_rack, 0, 1, 0)

In [56]:
ber.pickUpNextTip()

In [59]:
ber.move(z=50)

In [58]:
for sample in samples_list:
    ber.transferLiquid(sample, waste, 200)

In [95]:
ber.dumpTipToWaste()

# One-time settings

## Parameters for the beads tube and sample type

In [47]:
t2 = bl.sample_type('2ml')

In [48]:
t2._setZAboveRacks({})

In [49]:
r = ber.samples_rack

In [50]:
t2.setZAboveSpecificRack(r, 20.0)

In [51]:
t2.getZAboveSpecificRack(r)

20.0

In [52]:
t2.setInnerDiameter(8.30)

In [ ]:
# Freshly measured
depth_to_vol_dict = {
    14000: 13.24,
    10000: 38.0,
    5000: 71.0,
    2000: 91.8,
    1600: 93.9,
    1400: 95.7,
    1200: 96.8,
    1000: 98.8,
    800: 99.0,
    600: 103.0,
    400: 105.45,
    200: 109.0,
    0: 45.45,   
}

In [21]:
ber.calibrateRack(rack='tips')

(77.4, 149.965, 169.4)